# Live Weapon Detection

In [1]:
import xml.etree.ElementTree as ET
import os
import pandas as pd
from skimage.transform import resize
import numpy as np
import random
import math
from PIL import Image
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from plotnine import *
from torchvision.io.image import decode_image
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2, FasterRCNN_ResNet50_FPN_V2_Weights
from torchvision.utils import draw_bounding_boxes
from torchvision.transforms.functional import to_pil_image
from torchvision import transforms
from torchvision import datasets
import torch
import shutil

In [30]:
# create a csv: filename, width, height, depth, xmin, ymin, xmax, ymax
text = ''
def write_to_csv(filepath:str, output_path:str) -> None:
    """

    Writes elements from an XML file to a CSV file.

    Parameters:
        filepath (str): Filepath pointing to the XML file
        output_path (str): Name of the CSV file to write

    Returns:
        None

    """
    tree = ET.parse(filepath)
    root = tree.getroot()
        
    # write contents
    text = ''
    for child in root:
        if child.tag == 'filename':
            text += str(child.text)
        if child.tag == 'size':
            # 0 = width, 1 = height, 2 = depth
            for i in range(3):
                text += ',' + str(child[i].text)
        if child.tag == 'object':
            text += ',' + str(child[0].text)
            # 0 = xmin, 1 = ymin, 2 = xmax, 3 = ymax
            for i in range(4):
                text += ',' + str(child[4][i].text)
            # There may be multiple bounding boxes
            # For now, only consider the first one
            break

    with open(output_path, 'a') as csvfile:
        csvfile.write(text)
        csvfile.write('\n')

In [15]:
def resize_images(images_path:str, resized_width:int, resized_height:int, save=False) -> tuple:
    """
    
    Resizes images to a given width and height.

    Parameters:
        images_path (str): Filepath containing the images to resize.
        resized_width (int): Width to resize the images.
        resized_height (int): Height to resize the images.

    Returns:
        tuple: NumPy arrays containing the resized images and generated labels.
    
    """
    if save:
        os.makedirs('datasets/images_resized/', exist_ok=True)
    # Resize images
    resized_images = []
    labels = []
    count = 0
    for filename in os.listdir(images_path):
        file_path = os.path.join(images_path, filename)
        if os.path.isfile(file_path):
            with Image.open(file_path) as img:
                # With Scikit-Image:
                # img_data = np.asarray(img)
                # resized = resize(img_data, (resized_width, resized_height))
                # resized_images.append(resized)

                # With PyTorch:
                resize_transform = transforms.Resize((resized_height, resized_width))
                resized = resize_transform(img)

                if save:
                    # Saves resized images
                    resized.save('datasets/images_resized/' + filename)
                    count += 1
                    
                resized_images.append(resized)

    return resized_images

In [31]:
with open('weapons.csv', 'a') as csvfile:
    header = 'Filename,Width,Height,Depth,Name,Xmin,Ymin,Xmax,Ymax'
    csvfile.write(header)
    csvfile.write('\n')

with open('weapons_test.csv', 'a') as csvfile:
    header = 'Filename,Width,Height,Depth,Name,Xmin,Ymin,Xmax,Ymax'
    csvfile.write(header)
    csvfile.write('\n')

path = 'datasets/Sohas_weapon-Detection/annotations/xmls/'

for filename in os.listdir(path):
    file_path = os.path.join(path, filename)
    if os.path.isfile(file_path):
        write_to_csv(file_path, 'weapons.csv')

test_path = 'datasets/Sohas_weapon-Detection/annotations_test/xmls/'

for filename in os.listdir(test_path):
    file_path = os.path.join(test_path, filename)
    if os.path.isfile(file_path):
        write_to_csv(file_path, 'weapons_test.csv')

In [32]:
# Get min dimensions
weapons_data = pd.concat(
    map(pd.read_csv, ['weapons.csv', 'weapons_test.csv']), ignore_index=True
)
min_width = weapons_data['Width'].min()
min_height = weapons_data['Height'].min()

In [17]:
# Saves resized images
images_path = 'datasets/images_full'
img = resize_images(images_path, min_width, min_height, save=True)

In [18]:
# Get resized image data
images_path ='datasets/images_resized'
resized_images = resize_images(images_path, min_width, min_height)

In [103]:
tensor_images = []
for img in resized_images:
    to_tensor = transforms.ToTensor()
    image = to_tensor(img)
    tensor_images.append(image)
tensor_images = np.array(tensor_images)
tensor_images

array([[[[0.09019608, 0.08627451, 0.08627451, ..., 0.5058824 ,
          0.5058824 , 0.5058824 ],
         [0.09019608, 0.08627451, 0.08627451, ..., 0.5058824 ,
          0.5058824 , 0.5058824 ],
         [0.08627451, 0.08627451, 0.08235294, ..., 0.5058824 ,
          0.5058824 , 0.5058824 ],
         ...,
         [0.09019608, 0.09411765, 0.09803922, ..., 0.34117648,
          0.3372549 , 0.3372549 ],
         [0.04705882, 0.05098039, 0.05882353, ..., 0.27450982,
          0.27058825, 0.27058825],
         [0.11764706, 0.12156863, 0.1254902 , ..., 0.13725491,
          0.13725491, 0.13725491]],

        [[0.09411765, 0.09019608, 0.09019608, ..., 0.49803922,
          0.49803922, 0.49803922],
         [0.09411765, 0.09019608, 0.09019608, ..., 0.49803922,
          0.49803922, 0.49803922],
         [0.09019608, 0.09019608, 0.08627451, ..., 0.49803922,
          0.49803922, 0.49803922],
         ...,
         [0.08235294, 0.08627451, 0.09019608, ..., 0.3647059 ,
          0.36078432, 0.3

## Prepare Image Data for the Model

In [110]:
class ImageDataset(torch.utils.data.IterableDataset):
    def __init__(self, data):
        super().__init__()
        self.data = data

    def __iter__(self):
        for item in self.data:
            yield item

In [113]:
image_data = ImageDataset(tensor_images)
random.seed(72)
loader = torch.utils.data.DataLoader(dataset=image_data, batch_size=10)

In [19]:
src = 'datasets/images_resized/'
train_dest = os.path.join(src, 'train')
os.makedirs(train_dest, exist_ok=True)
val_dest = os.path.join(src, 'val')
os.makedirs(val_dest, exist_ok=True)

num_images = 1472
train_percentage = 0.75
num_train = int(num_images * train_percentage)
num_val = num_images - num_train

count = 0
for filename in os.listdir(src):
    file_path = os.path.join(src, filename)
    if os.path.isfile(file_path) and count < num_train: 
        # Ensure the file is a file and only move images up to train percentage
        shutil.move(file_path, train_dest)
    count += 1

count = 0
for filename in os.listdir(src):
    file_path = os.path.join(src, filename)
    if os.path.isfile(file_path) and count < num_val:
        shutil.move(file_path, val_dest)
    count += 1

In [ ]:
# Create a classes/labels subdirectory that specify if the image has a weapon or not
for filename in os.listdir(train_dest):
    # Get the corresponding image file with the same name
    results = weapons_data[weapons_data['Filename'] == filename]
    # Check if the image contains a weapon
    print(results['Name'].values)
    

knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
knife
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
billete
bill

IndexError: index 0 is out of bounds for axis 0 with size 0

Also found this pipeline on https://docs.pytorch.org/tutorials/beginner/transfer_learning_tutorial.html for transforming images. I may go with this as opposed to the resize_images method above.

In [ ]:
def transform_images(resized_width, resized_height, data_dir):
    data_transforms = {
        'train': transforms.Compose([
            transforms.Resize((resized_height, resized_width)),
            transforms.ToTensor()
        ]),
        'val': transforms.Compose([
            transforms.Resize((resized_height, resized_width)),
            transforms.ToTensor()
        ])
    }

    image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
    dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=10, shuffle=True, num_workers=4) for x in ['train', 'val']}
    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
    class_names = image_datasets['train'].classes

    return dataloaders

In [54]:
dataloaders = transform_images(min_width, min_height, 'datasets/images_resized/')
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

FileNotFoundError: Couldn't find any class folder in datasets/images_resized/train.